In [14]:
ls

command.sh  floyd_requirements.txt  layer1rnnGPU.ipynb


In [15]:
ls /input

ls: cannot access '/input': No such file or directory


In [16]:
#!pip install nltk

In [17]:
!pip install gensim

In [7]:
!pip install --upgrade Keras

Requirement already up-to-date: Keras in /usr/local/lib/python3.5/site-packages
Requirement already up-to-date: theano in /usr/local/lib/python3.5/site-packages (from Keras)
Requirement already up-to-date: pyyaml in /usr/local/lib/python3.5/site-packages (from Keras)
Requirement already up-to-date: six in /usr/local/lib/python3.5/site-packages (from Keras)
Requirement already up-to-date: numpy>=1.9.1 in /usr/local/lib/python3.5/site-packages (from theano->Keras)
Requirement already up-to-date: scipy>=0.14 in /usr/local/lib/python3.5/site-packages (from theano->Keras)


In [18]:
!pip install h5py

In [19]:
#!pip install spacy
#from spacy.en import English
#nlp = English()

In [20]:
#!wget http://nlp.stanford.edu/data/glove.840B.300d.zip

In [21]:
!wget https://dl.dropboxusercontent.com/s/pvyz8lewd7lyuum/GoogleNews-vectors-negative300.zip

--2017-05-25 15:17:19--  https://dl.dropboxusercontent.com/s/pvyz8lewd7lyuum/GoogleNews-vectors-negative300.zip
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.1.6
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.1.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1282016977 (1.2G) [application/zip]
Saving to: ‘GoogleNews-vectors-negative300.zip’

GoogleNews-vectors- 100%[===================>]   1.19G  58.5MB/s    in 24s     

2017-05-25 15:17:52 (50.4 MB/s) - ‘GoogleNews-vectors-negative300.zip’ saved [1282016977/1282016977]



In [22]:
!wget https://dl.dropboxusercontent.com/s/tnmz8pdkoqfawg6/test_spacyLemma.csv

--2017-05-25 15:17:52--  https://dl.dropboxusercontent.com/s/tnmz8pdkoqfawg6/test_spacyLemma.csv
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.1.6
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.1.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 320217523 (305M) [text/csv]
Saving to: ‘test_spacyLemma.csv’

test_spacyLemma.csv 100%[===================>] 305.38M  43.2MB/s    in 6.7s    

2017-05-25 15:18:01 (45.4 MB/s) - ‘test_spacyLemma.csv’ saved [320217523/320217523]



In [23]:
!wget https://dl.dropboxusercontent.com/s/tjln8kttfffb1gi/train_spacyLemma.csv

--2017-05-25 15:18:01--  https://dl.dropboxusercontent.com/s/tjln8kttfffb1gi/train_spacyLemma.csv
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.1.6
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.1.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60869523 (58M) [text/csv]
Saving to: ‘train_spacyLemma.csv’

train_spacyLemma.cs 100%[===================>]  58.05M  67.5MB/s    in 0.9s    

2017-05-25 15:18:07 (67.5 MB/s) - ‘train_spacyLemma.csv’ saved [60869523/60869523]



In [24]:
#!rm train_spacyLemma.csv?dl=0

In [25]:
!unzip GoogleNews-vectors-negative300.zip

Archive:  GoogleNews-vectors-negative300.zip
 bunzipping: GoogleNews-vectors-negative300.bin  


In [26]:
ls -l --block-size=M

total 5062M
-rw-r--r-- 1 root root 3476M Dec 11  2013 GoogleNews-vectors-negative300.bin
-rw-r--r-- 1 root root 1223M May 25 15:17 GoogleNews-vectors-negative300.zip
-rw-r--r-- 1 root root    1M May 25 15:07 command.sh
-rw-r--r-- 1 root root    1M May 25 15:07 floyd_requirements.txt
-rw-r--r-- 1 root root    1M May 25 15:20 layer1rnnGPU.ipynb
-rw-r--r-- 1 root root  306M May 25 15:18 test_spacyLemma.csv
-rw-r--r-- 1 root root   59M May 25 15:18 train_spacyLemma.csv


In [1]:
preprocessing = "_MyMagic3_spacyLemma"


import numpy as np
import pandas as pd
import operator
from sklearn.model_selection import train_test_split
from collections import Counter
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from pylab import plot, show, subplot, specgram, imshow, savefig
from sklearn.model_selection import KFold
K = 5
kf = KFold(n_splits = K)
#==============================================================================
# print(list(enumerate(kf.split([1,2,3,4,5,6,7,8,9]))))   
# X= np.array(["a","b","c","d","e","f","g","h","i"])
# for kth, (train_index, test_index) in enumerate(kf.split(X)):
#     print(test_index)
#     max = np.amax(test_index)+1
#     min = np.amin(test_index)
#     print(X[min:max])
#==============================================================================

np.set_printoptions(threshold=400000)
pd.set_option('display.max_rows', 2000, 'display.max_columns', 2000,  'display.show_dimensions', 'truncate')


RS = 12357

In [2]:
import re
import csv
import codecs
import numpy as np
import pandas as pd

from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Bidirectional, GRU
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [3]:
########################################
## set directories and parameters
BASE_DIR = 'F:/DS-main/Kaggle-main/Quora Question Pairs - inputs/data/'
EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin'
TRAIN_DATA_FILE = 'train_spacyLemma.csv'
TEST_DATA_FILE = 'test_spacyLemma.csv'
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

num_lstm = 300
num_dense = 200
rate_drop_lstm = 0.3
rate_drop_dense = 0.3

act = 'relu'
re_weight = True # whether to re-weight classes to fit the 17.5% share in test set

STAMP = 'lstm_%d_%d_%.2f_%.2f'%(num_lstm, num_dense, rate_drop_lstm, \
        rate_drop_dense)

In [4]:
## index word vectors
print('Indexing word vectors')

word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, \
        binary=True)
print('Found %s word vectors of word2vec' % len(word2vec.vocab))

Indexing word vectors
Found 3000000 word vectors of word2vec


In [5]:
## process texts in datasets
print('Processing text dataset')

# The function "text_to_wordlist" is from
# https://www.kaggle.com/currie32/quora-question-pairs/the-importance-of-cleaning-text
def text_to_wordlist(text):
    # Clean the text, with the option to remove stopwords and to stem words.
    if text == "":
        text = "empty"
    
    # Clean the text
    text = re.sub("？", "?", text) 
    text = re.sub("…", "", text) 
    text = re.sub("é", "e", text) 
    text = re.sub(r" quikly ", " quickly ", text)
    text = re.sub(r" unseccessful ", " unsuccessful ", text)
    text = re.sub(r" addmision ", " admission ", text)
    text = re.sub(r" insititute ", " institute ", text)
    text = re.sub(r" connectionn ", " connection ", text)
    text = re.sub(r" permantley ", " permanently ", text)
    text = re.sub(r" sylabus ", " syllabus ", text)
    text = re.sub(r" sequrity ", " security ", text)
    text = re.sub(r" latop", " laptop", text)
    text = re.sub(r" programmning ", " programming ", text)  
    text = re.sub(r" begineer ", " beginner ", text)  
    text = re.sub(r" qoura ", " Quora ", text)
    text = re.sub(r" wtiter ", " writer ", text)  
    text = re.sub(r" litrate ", " literate ", text)  
        
    # Return a list of words
    return(text)

texts_1 = [] 
texts_2 = []
labels = []
with codecs.open(TRAIN_DATA_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        texts_1.append(text_to_wordlist(values[3]))
        texts_2.append(text_to_wordlist(values[4]))
        labels.append(int(values[5]))
        if (len(texts_1)%100000 == 0 ):
            print ('proc train',len(texts_1))
print('Found %s texts in train.csv' % len(texts_1))

test_texts_1 = []
test_texts_2 = []
test_ids = []
with codecs.open(TEST_DATA_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        test_texts_1.append(text_to_wordlist(values[1]))
        test_texts_2.append(text_to_wordlist(values[2]))
        test_ids.append(values[0])
        if (len(test_texts_1)%100000 == 0 ):
            print ('test',len(test_texts_1))
print('Found %s texts in test.csv' % len(test_texts_1))

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts_1 + texts_2 + test_texts_1 + test_texts_2)

sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)
test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)
test_sequences_2 = tokenizer.texts_to_sequences(test_texts_2)

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

data_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
data_2 = pad_sequences(sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(labels)
print('Shape of data tensor:', data_1.shape)
print('Shape of label tensor:', labels.shape)

test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
test_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
test_ids = np.array(test_ids)

del sequences_1, sequences_2, test_sequences_1, test_sequences_2, texts_1, texts_2, test_texts_1, test_texts_2

Processing text dataset
proc train 100000
proc train 200000
proc train 300000
proc train 400000
Found 404290 texts in train.csv
test 100000
test 200000
test 300000
test 400000
test 500000
test 600000
test 700000
test 800000
test 900000
test 1000000
test 1100000
test 1200000
test 1300000
test 1400000
test 1500000
test 1600000
test 1700000
test 1800000
test 1900000
test 2000000
test 2100000
test 2200000
test 2300000
Found 2345796 texts in test.csv
Found 116309 unique tokens
Shape of data tensor: (404290, 30)
Shape of label tensor: (404290,)


In [6]:
## prepare embeddings
print('Preparing embedding matrix')

nb_words = min(MAX_NB_WORDS, len(word_index))+1

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix
Null word embeddings: 65681


In [ ]:
## sample train/validation data

data_1_dbl = np.vstack((data_1, data_2))
data_2_dbl = np.vstack((data_2, data_1))
labels_dbl = np.concatenate((labels, labels))


stacking_train = np.empty(shape=(len(data_1_dbl)))
stacking_test = np.empty(shape=(len(test_data_1),K))
for kth, (train_index, test_index) in enumerate(kf.split(data_1_dbl)):    
    print("\n********************** FOLD ", kth+1, " ******************\n")
    max_idx_test  = np.amax(test_index)+1
    min_idx_test  = np.amin(test_index)
    max_idx_train = np.amax(train_index)+1
    min_idx_train = np.amin(train_index)
    
    
    weight_val = np.ones(len(labels_dbl[min_idx_test:max_idx_test]))
    if re_weight:
        weight_val *= 0.472001959
        weight_val[labels[min_idx_test:max_idx_test]==0] = 1.309028344
        
    ########################################
    ## define the model structure
    model = Sequential()
    
    embedding_layer = Embedding(nb_words,
            EMBEDDING_DIM,
            weights=[embedding_matrix],
            input_length=MAX_SEQUENCE_LENGTH,
            trainable=False)
    #lstm_layer0 = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm, return_sequences=True)
    gru_layer = GRU(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)
    #lstm_layer1 = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)
    #lstm_layer = Bidirectional(GRU(num_lstm, return_sequences=True))
    
    
    sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences_1 = embedding_layer(sequence_1_input)
    x1 = gru_layer(embedded_sequences_1)
    #x2 = lstm_layer1(x1)
    
    sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences_2 = embedding_layer(sequence_2_input)
    y1 = gru_layer(embedded_sequences_2)
    #y2 = lstm_layer1(y1)
    
    merged = concatenate([x1, y1])
    #Imerged = concatenate([x2, y2])
    merged = Dropout(rate_drop_dense)(merged)
    merged = BatchNormalization()(merged)
    
    merged = Dense(num_dense, activation=act)(merged)
    merged = Dropout(rate_drop_dense)(merged)
    merged = BatchNormalization()(merged)
    
    preds = Dense(1, activation='sigmoid')(merged)
    
    ########################################
    ## add class weight
    if re_weight:
        class_weight = {0: 1.309028344, 1: 0.472001959}
    else:
        class_weight = None
    
    ########################################
    ## train the model
    model = Model(inputs=[sequence_1_input, sequence_2_input], \
            outputs=preds)
    model.compile(loss='binary_crossentropy',
            optimizer='nadam',
            metrics=['acc'])
    #model.summary()
    print(STAMP)
    
    early_stopping =EarlyStopping(monitor='val_loss', patience=5)
    bst_model_path = STAMP + '.h5'
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True, verbose=1)
    
    batch_size = 512

    hist = model.fit([data_1_dbl[min_idx_train:max_idx_train], data_2_dbl[min_idx_train:max_idx_train]], labels_dbl[min_idx_train:max_idx_train], \
        validation_data=([data_1_dbl[min_idx_test:max_idx_test], data_2_dbl[min_idx_test:max_idx_test]], labels_dbl[min_idx_test:max_idx_test], weight_val), \
        epochs=1000, batch_size=batch_size, shuffle=True, \
        class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])
    model.load_weights(bst_model_path)
    bst_val_score = min(hist.history['val_loss'])
       #predict training ids of the testing fold
    
    stacking_train[min_idx_test:max_idx_test] = model.predict([data_1_dbl[min_idx_test:max_idx_test], data_2_dbl[min_idx_test:max_idx_test]], batch_size=batch_size, verbose=1)[:,0]
    stacking_train[min_idx_test:max_idx_test] += model.predict([data_2_dbl[min_idx_test:max_idx_test], data_1_dbl[min_idx_test:max_idx_test]], batch_size=batch_size, verbose=1)[:,0]
    stacking_train[min_idx_test:max_idx_test] /= 2
    #pred test
    stacking_test[:,kth] = model.predict([test_data_1, test_data_2], batch_size=batch_size, verbose=1)[:,0]
    stacking_test[:,kth] += model.predict([test_data_2, test_data_1], batch_size=batch_size, verbose=1)[:,0]
    stacking_test[:,kth] /= 2




********************** FOLD  1  ******************

lstm_300_200_0.30_0.30
Train on 646864 samples, validate on 161716 samples
Epoch 1/1000
646864/646864 [==============================] - 227s - loss: 0.3912 - acc: 0.6880 - val_loss: 0.3420 - val_acc: 0.7485
Epoch 2/1000
646864/646864 [==============================] - 226s - loss: 0.3372 - acc: 0.7280 - val_loss: 0.3057 - val_acc: 0.7611
Epoch 3/1000
646864/646864 [==============================] - 226s - loss: 0.3176 - acc: 0.7474 - val_loss: 0.2947 - val_acc: 0.7765
Epoch 4/1000
646864/646864 [==============================] - 226s - loss: 0.3057 - acc: 0.7595 - val_loss: 0.2848 - val_acc: 0.8028
Epoch 5/1000
646864/646864 [==============================] - 226s - loss: 0.2968 - acc: 0.7680 - val_loss: 0.2719 - val_acc: 0.8051
Epoch 6/1000
646864/646864 [==============================] - 225s - loss: 0.2903 - acc: 0.7750 - val_loss: 0.2735 - val_acc: 0.8193
Epoch 7/1000
646864/646864 [==============================] - 226s - loss:

646864/646864 [==============================] - 225s - loss: 0.2552 - acc: 0.8110 - val_loss: 0.2298 - val_acc: 0.8596
Epoch 29/1000
646864/646864 [==============================] - 227s - loss: 0.2551 - acc: 0.8112 - val_loss: 0.2237 - val_acc: 0.8587
Epoch 30/1000
646864/646864 [==============================] - 227s - loss: 0.2546 - acc: 0.8120 - val_loss: 0.2216 - val_acc: 0.8456
Epoch 31/1000
646864/646864 [==============================] - 225s - loss: 0.2536 - acc: 0.8131 - val_loss: 0.2295 - val_acc: 0.8614
Epoch 32/1000
646864/646864 [==============================] - 225s - loss: 0.2531 - acc: 0.8132 - val_loss: 0.2271 - val_acc: 0.8619
Epoch 33/1000
646864/646864 [==============================] - 227s - loss: 0.2531 - acc: 0.8133 - val_loss: 0.2211 - val_acc: 0.8618
Epoch 34/1000
646864/646864 [==============================] - 225s - loss: 0.2528 - acc: 0.8137 - val_loss: 0.2226 - val_acc: 0.8595
Epoch 35/1000
646864/646864 [==============================] - 227s - loss: 

646864/646864 [==============================] - 227s - loss: 0.2450 - acc: 0.8215 - val_loss: 0.2097 - val_acc: 0.8623
Epoch 58/1000
646864/646864 [==============================] - 225s - loss: 0.2451 - acc: 0.8212 - val_loss: 0.2163 - val_acc: 0.8704
Epoch 59/1000
646864/646864 [==============================] - 225s - loss: 0.2447 - acc: 0.8216 - val_loss: 0.2198 - val_acc: 0.8735
Epoch 60/1000
646864/646864 [==============================] - 225s - loss: 0.2445 - acc: 0.8223 - val_loss: 0.2121 - val_acc: 0.8679
Epoch 61/1000
646864/646864 [==============================] - 225s - loss: 0.2438 - acc: 0.8229 - val_loss: 0.2103 - val_acc: 0.8720
Epoch 62/1000
646864/646864 [==============================] - 225s - loss: 0.2438 - acc: 0.8227 - val_loss: 0.2122 - val_acc: 0.8731
Epoch 63/1000
2345472/2345796 [============================>.] - ETA: 0s
********************** FOLD  2  ******************

lstm_300_200_0.30_0.30
Train on 808580 samples, validate on 161716 samples
Epoch 1/10

808580/808580 [==============================] - 277s - loss: 0.2571 - acc: 0.8088 - val_loss: 0.2091 - val_acc: 0.8767
Epoch 23/1000
808580/808580 [==============================] - 278s - loss: 0.2569 - acc: 0.8092 - val_loss: 0.1996 - val_acc: 0.8713
Epoch 24/1000
808580/808580 [==============================] - 278s - loss: 0.2557 - acc: 0.8106 - val_loss: 0.1987 - val_acc: 0.8755
Epoch 25/1000
808580/808580 [==============================] - 277s - loss: 0.2547 - acc: 0.8113 - val_loss: 0.2040 - val_acc: 0.8824
Epoch 26/1000
808580/808580 [==============================] - 277s - loss: 0.2547 - acc: 0.8117 - val_loss: 0.1995 - val_acc: 0.8758
Epoch 27/1000
808580/808580 [==============================] - 277s - loss: 0.2540 - acc: 0.8119 - val_loss: 0.1998 - val_acc: 0.8739
Epoch 28/1000
808580/808580 [==============================] - 278s - loss: 0.2537 - acc: 0.8122 - val_loss: 0.1965 - val_acc: 0.8774
Epoch 29/1000
808580/808580 [==============================] - 277s - loss: 

808580/808580 [==============================] - 277s - loss: 0.2463 - acc: 0.8196 - val_loss: 0.1880 - val_acc: 0.8896
Epoch 52/1000
808580/808580 [==============================] - 277s - loss: 0.2461 - acc: 0.8200 - val_loss: 0.1868 - val_acc: 0.8902
Epoch 53/1000
808580/808580 [==============================] - 277s - loss: 0.2467 - acc: 0.8195 - val_loss: 0.1869 - val_acc: 0.8835
Epoch 54/1000
808580/808580 [==============================] - 277s - loss: 0.2460 - acc: 0.8201 - val_loss: 0.1886 - val_acc: 0.8912
Epoch 55/1000
808580/808580 [==============================] - 277s - loss: 0.2485 - acc: 0.8180 - val_loss: 0.1866 - val_acc: 0.8863
Epoch 56/1000
2345472/2345796 [============================>.] - ETA: 0s
********************** FOLD  3  ******************

lstm_300_200_0.30_0.30
Train on 808580 samples, validate on 161716 samples
Epoch 1/1000
808580/808580 [==============================] - 280s - loss: 0.3844 - acc: 0.6926 - val_loss: 0.2779 - val_acc: 0.7548
Epoch 2/100

808580/808580 [==============================] - 278s - loss: 0.2924 - acc: 0.7740 - val_loss: 0.1652 - val_acc: 0.8307
Epoch 6/1000
808580/808580 [==============================] - 278s - loss: 0.2864 - acc: 0.7798 - val_loss: 0.1603 - val_acc: 0.8362
Epoch 7/1000
808580/808580 [==============================] - 278s - loss: 0.2823 - acc: 0.7840 - val_loss: 0.1602 - val_acc: 0.8329
Epoch 8/1000
808580/808580 [==============================] - 278s - loss: 0.2785 - acc: 0.7880 - val_loss: 0.1535 - val_acc: 0.8435
Epoch 9/1000
808580/808580 [==============================] - 278s - loss: 0.2755 - acc: 0.7908 - val_loss: 0.1513 - val_acc: 0.8454
Epoch 10/1000
808580/808580 [==============================] - 278s - loss: 0.2730 - acc: 0.7935 - val_loss: 0.1483 - val_acc: 0.8525
Epoch 11/1000
808580/808580 [==============================] - 276s - loss: 0.2718 - acc: 0.7946 - val_loss: 0.1559 - val_acc: 0.8402
Epoch 12/1000
808580/808580 [==============================] - 278s - loss: 0.27

808580/808580 [==============================] - 283s - loss: 0.2543 - acc: 0.8125 - val_loss: 0.1238 - val_acc: 0.8840
Epoch 34/1000
808580/808580 [==============================] - 282s - loss: 0.2541 - acc: 0.8125 - val_loss: 0.1277 - val_acc: 0.8779
Epoch 35/1000
808580/808580 [==============================] - 285s - loss: 0.2537 - acc: 0.8128 - val_loss: 0.1280 - val_acc: 0.8763
Epoch 36/1000
808580/808580 [==============================] - 286s - loss: 0.2532 - acc: 0.8132 - val_loss: 0.1282 - val_acc: 0.8760
Epoch 37/1000
808580/808580 [==============================] - 276s - loss: 0.2524 - acc: 0.8139 - val_loss: 0.1244 - val_acc: 0.8828
Epoch 38/1000
808580/808580 [==============================] - 276s - loss: 0.2521 - acc: 0.8141 - val_loss: 0.1255 - val_acc: 0.8794
Epoch 39/1000
2345472/2345796 [============================>.] - ETA: 0s
********************** FOLD  5  ******************

lstm_300_200_0.30_0.30
Train on 646864 samples, validate on 161716 samples
Epoch 1/10

646864/646864 [==============================] - 227s - loss: 0.2591 - acc: 0.8071 - val_loss: 0.1487 - val_acc: 0.8516
Epoch 23/1000
646864/646864 [==============================] - 227s - loss: 0.2577 - acc: 0.8079 - val_loss: 0.1424 - val_acc: 0.8631
Epoch 24/1000
646864/646864 [==============================] - 226s - loss: 0.2571 - acc: 0.8088 - val_loss: 0.1464 - val_acc: 0.8540
Epoch 25/1000
646864/646864 [==============================] - 226s - loss: 0.2562 - acc: 0.8103 - val_loss: 0.1470 - val_acc: 0.8530
Epoch 26/1000
646864/646864 [==============================] - 226s - loss: 0.2557 - acc: 0.8106 - val_loss: 0.1486 - val_acc: 0.8495
Epoch 27/1000
646864/646864 [==============================] - 226s - loss: 0.2547 - acc: 0.8115 - val_loss: 0.1432 - val_acc: 0.8595
Epoch 28/1000
646864/646864 [==============================] - 226s - loss: 0.2540 - acc: 0.8123 - val_loss: 0.1475 - val_acc: 0.8521
Epoch 29/1000
2345472/2345796 [============================>.] - ETA: 0s

In [ ]:

#Saving 
#pred test
preds = (stacking_test[:,0] + stacking_test[:,1] + stacking_test[:,2] + stacking_test[:,3] + stacking_test[:,4])/5
print("Writing output...")
sub = pd.DataFrame()
sub['test_id'] = df_test['test_id']
sub['is_duplicate'] = preds
sub.to_csv("GRU_test_kf" + preprocessing  + ".csv", index=False)

#--- pred training for ensemble
print("Writing training pred output...")
sub = pd.DataFrame()
sub['is_duplicate'] = stacking_train
sub.to_csv("GRU_train_kf" + preprocessing  + ".csv", index=False)


In [ ]:
!ls